In [14]:
import ctypes
import sys
import os 
import numpy as np
from ctypes import POINTER

In [15]:
cwd =  os.getcwd()

In [16]:
lib_h = ctypes.CDLL(cwd + "/friedmann_Cs_lib.so") 

In [41]:
#double omega_dm0,double B1,double sig8,double *data,double *fs8,int n,int ax=1,int print_file=0
lib_h.bimetric_fs8_log_like.argtypes = [ctypes.c_double,ctypes.c_double,ctypes.c_double,POINTER(ctypes.c_double),POINTER(ctypes.c_double),ctypes.c_int,ctypes.c_int,ctypes.c_int]
lib_h.restype = ctypes.c_int

In [50]:
u = np.array( (0.02,0.08,0.15,0.19,0.23,0.35,0.45,0.55,0.65,0.78,0.89,0.999))
n = int(u.shape[0])
zz = ctypes.c_double*(n*2)
z = zz(0.02,0.0,0.08,0.0,0.15,0.0,0.19,0.0,0.23,0.0,0.35,0.0,0.45,0.0,0.55,0.0,0.65,0.0,0.78,0.0,0.89,0.0,0.999,0.0)
fs8 = ctypes.c_double*n
fs8a = fs8(0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)


In [53]:
l = lib_h.bimetric_fs8_log_like(0.3,0.0,0.8,z,fs8a,n,1,1)

In [61]:
dg = np.ones((18,))

In [62]:
m = np.diagflat(dg)

In [68]:
mi = np.linalg.inv(m)

array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0